<a href="https://colab.research.google.com/github/hamed-jamali-software/mp32face/blob/main/coqui_XTTS_v2_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content
!git clone -b dev https://github.com/camenduru/xtts2-hf
%cd /content/xtts2-hf
!pip install -q gradio==3.50.2 TTS==0.21.1 langid unidic-lite unidic deepspeed
!pip install -q numpy<2.0.0 -U
!wget https://huggingface.co/spaces/coqui/xtts/resolve/main/examples/female.wav -O /content/xtts2-hf/examples/female.wav
!wget https://huggingface.co/spaces/coqui/xtts/resolve/main/examples/male.wav -O /content/xtts2-hf/examples/male.wav
!wget https://huggingface.co/spaces/coqui/xtts/resolve/main/ffmpeg.zip -O /content/xtts2-hf/ffmpeg.zip


/content
Cloning into 'xtts2-hf'...
remote: Enumerating objects: 518, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 518 (delta 2), reused 3 (delta 1), pack-reused 512 (from 1)
Receiving objects: 100% (518/518), 716.12 KiB | 13.26 MiB/s, done.
Resolving deltas: 100% (311/311), done.
Error downloading object: examples/female.wav (89a4fa9): Smudge error: Error downloading examples/female.wav (89a4fa9a16b6463f852cf9424f72c3d3c87aa83010e89db534c53fcd1ae12c02): [89a4fa9a16b6463f852cf9424f72c3d3c87aa83010e89db534c53fcd1ae12c02] Object does not exist on the server: [404] Object does not exist on the server

Errors logged to /content/xtts2-hf/.git/lfs/logs/20240829T123812.905244375.log
Use `git lfs logs last` to view the log.
error: external filter 'git-lfs filter-process' failed
fatal: examples/female.wav: smudge filter lfs failed
You can inspect what was checked out with 'git status'
and retry with 'git restore --source=HEAD :/'

In [ ]:
!python app.py

In [2]:
import sys
import io, os, stat
import subprocess
import random
from zipfile import ZipFile
import uuid
import time
import torch
import torchaudio
import subprocess
import uuid
import time
import torch
import torchaudio
import langid
import re
from zipfile import ZipFile
from TTS.api import TTS
from TTS.tts.configs.xtts_config import XttsConfig
from TTS.tts.models.xtts import Xtts
from TTS.utils.generic_utils import get_user_data_dir
from huggingface_hub import HfApi


In [3]:

# Download for mecab
os.system('python -m unidic download')

# By using XTTS you agree to CPML license https://coqui.ai/cpml
os.environ["COQUI_TOS_AGREED"] = "1"

HF_TOKEN = os.environ.get("HF_TOKEN")
api = HfApi(token=HF_TOKEN)
repo_id = "coqui/xtts"

# Use newer ffmpeg binary for Ubuntu20 to use denoising for microphone input
print("Export newer ffmpeg binary for denoise filter")
ZipFile("ffmpeg.zip").extractall()
print("Make ffmpeg binary executable")
st = os.stat("ffmpeg")
os.chmod("ffmpeg", st.st_mode | stat.S_IEXEC)

# This will trigger downloading model
print("Downloading if not downloaded Coqui XTTS V2")
from TTS.utils.manage import ModelManager

model_name = "tts_models/multilingual/multi-dataset/xtts_v2"
ModelManager().download_model(model_name)
model_path = os.path.join(get_user_data_dir("tts"), model_name.replace("/", "--"))
print("XTTS downloaded")

Export newer ffmpeg binary for denoise filter
Make ffmpeg binary executable
 > Downloading model to /root/.local/share/tts/tts_models--multilingual--multi-dataset--xtts_v2
 > Model's license - CPML
 > Check https://coqui.ai/cpml.txt for more info.
XTTS downloaded


In [4]:

config = XttsConfig()
config.load_json(os.path.join(model_path, "config.json"))

model = Xtts.init_from_config(config)
model.load_checkpoint(
    config,
    checkpoint_path=os.path.join(model_path, "model.pth"),
    vocab_path=os.path.join(model_path, "vocab.json"),
    eval=True,
    use_deepspeed=True,
)
model.cuda()

DEVICE_ASSERT_DETECTED = 0
DEVICE_ASSERT_PROMPT = None
DEVICE_ASSERT_LANG = None

supported_languages = config.languages


/usr/local/lib/python3.10/dist-packages/TTS/utils/io.py:54: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=map_location, **kwargs)


[2024-08-29 12:42:37,012] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-08-29 12:42:39,722] [INFO] [logging.py:96:log_dist] [Rank -1] DeepSpeed info: version=0.15.0, git-hash=unknown, git-branch=unknown
[2024-08-29 12:42:39,724] [WARNING] [config_utils.py:70:_process_deprecated_field] Config parameter replace_method is deprecated. This parameter is no longer needed, please remove from your call to DeepSpeed-inference
[2024-08-29 12:42:39,728] [WARNING] [config_utils.py:70:_process_deprecated_field] Config parameter mp_size is deprecated use tensor_parallel.tp_size instead
[2024-08-29 12:42:39,729] [INFO] [logging.py:96:log_dist] [Rank -1] quantize_bits = 8 mlp_extra_grouping = False, quantize_groups = 1


Using /root/.cache/torch_extensions/py310_cu121 as PyTorch extensions root...
Creating extension directory /root/.cache/torch_extensions/py310_cu121/transformer_inference...
Detected CUDA files, patching ldflags
Emitting ninja build file /root/.cache/torch_extensions/py310_cu121/transformer_inference/build.ninja...
/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:1965: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
Building extension module transformer_inference...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


Time to load transformer_inference op: 83.93317246437073 seconds
[2024-08-29 12:44:04,359] [INFO] [logging.py:96:log_dist] [Rank -1] DeepSpeed-Inference config: {'layer_id': 0, 'hidden_size': 1024, 'intermediate_size': 4096, 'heads': 16, 'num_hidden_layers': -1, 'dtype': torch.float32, 'pre_layer_norm': True, 'norm_type': <NormType.LayerNorm: 1>, 'local_rank': -1, 'stochastic_mode': False, 'epsilon': 1e-05, 'mp_size': 1, 'scale_attention': True, 'triangular_masking': True, 'local_attention': False, 'window_size': 1, 'rotary_dim': -1, 'rotate_half': False, 'rotate_every_two': True, 'return_tuple': True, 'mlp_after_attn': True, 'mlp_act_func_type': <ActivationFuncType.GELU: 1>, 'specialized_mode': False, 'training_mp_size': 1, 'bigscience_bloom': False, 'max_out_tokens': 1024, 'min_out_tokens': 1, 'scale_attn_by_inverse_layer_idx': False, 'enable_qkv_quantization': False, 'use_mup': False, 'return_single_tuple': False, 'set_empty_params': False, 'transposed_mode': False, 'use_triton': Fa

Loading extension module transformer_inference...


In [5]:

def text_to_speech(prompt, language, audio_file_pth=None, mic_file_path=None, use_mic=False, voice_cleanup=False, no_lang_auto_detect=False, agree=True):
    if agree:
        if language not in supported_languages:
            print(f"Language you put {language} in is not in our Supported Languages, please choose from dropdown")
            return None

        language_predicted = langid.classify(prompt)[0].strip()
        if language_predicted == "zh":
            language_predicted = "zh-cn"

        if len(prompt) > 15:
            if language_predicted != language and not no_lang_auto_detect:
                print(f"It looks like your text isn’t the language you chose , if you’re sure the text is the same language you chose, please check disable language auto-detection checkbox")
                return None

        if use_mic:
            if mic_file_path is not None:
                speaker_wav = mic_file_path
            else:
                print("Please record your voice with Microphone, or uncheck Use Microphone to use reference audios")
                return None
        else:
            speaker_wav = audio_file_pth

        # Filtering for microphone input
        lowpassfilter = denoise = trim = loudness = True
        lowpass_highpass = "lowpass=8000,highpass=75," if lowpassfilter else ""
        trim_silence = "areverse,silenceremove=start_periods=1:start_silence=0:start_threshold=0.02,areverse,silenceremove=start_periods=1:start_silence=0:start_threshold=0.02," if trim else ""

        if voice_cleanup:
            try:
                out_filename = speaker_wav + str(uuid.uuid4()) + ".wav"
                shell_command = f"./ffmpeg -y -i {speaker_wav} -af {lowpass_highpass}{trim_silence} {out_filename}".split(" ")
                subprocess.run(shell_command, capture_output=False, text=True, check=True)
                speaker_wav = out_filename
                print("Filtered microphone input")
            except subprocess.CalledProcessError:
                print("Error: failed filtering, use original microphone input")

        if len(prompt) < 2:
            print("Please give a longer prompt text")
            return None
        if len(prompt) > 200:
            print("Text length limited to 200 characters for this demo, please try shorter text. You can clone this space and edit code for your own usage")
            return None

        global DEVICE_ASSERT_DETECTED
        if DEVICE_ASSERT_DETECTED:
            global DEVICE_ASSERT_PROMPT
            global DEVICE_ASSERT_LANG
            print(f"Unrecoverable exception caused by language:{DEVICE_ASSERT_LANG} prompt:{DEVICE_ASSERT_PROMPT}")
            space = api.get_space_runtime(repo_id=repo_id)
            if space.stage != "BUILDING":
                api.restart_space(repo_id=repo_id)
            else:
                print("TRIED TO RESTART but space is building")

        try:
            prompt = re.sub("([^\x00-\x7F]|\w)(\.|\。|\?)", r"\1 \2\2", prompt)
            print("Generating new audio...")
            t0 = time.time()
            out = model.inference(
                prompt,
                language,
                *model.get_conditioning_latents(audio_path=speaker_wav, gpt_cond_len=30, gpt_cond_chunk_len=4, max_ref_length=60),
                repetition_penalty=5.0,
                temperature=0.75,
            )
            inference_time = time.time() - t0
            print(f"Time to generate audio: {round(inference_time * 1000)} milliseconds")
            real_time_factor = (time.time() - t0) / out['wav'].shape[-1] * 24000
            print(f"Real-time factor (RTF): {real_time_factor}")
            torchaudio.save("output.wav", torch.tensor(out["wav"]).unsqueeze(0), 24000)
            print("Audio saved to output.wav")

        except RuntimeError as e:
            if "device-side assert" in str(e):
                print(f"Exit due to: Unrecoverable exception caused by language:{language} prompt:{prompt}")
                print("Unhandled Exception encounter, please retry in a minute")
                print("Cuda device-assert Runtime encountered need restart")
                if not DEVICE_ASSERT_DETECTED:
                    DEVICE_ASSERT_DETECTED = 1
                    DEVICE_ASSERT_PROMPT = prompt
                    DEVICE_ASSERT_LANG = language

                error_time = datetime.datetime.now().strftime("%d-%m-%Y-%H:%M:%S")
                error_data = [
                    error_time,
                    prompt,
                    language,
                    audio_file_pth,
                    mic_file_path,
                    use_mic,
                    voice_cleanup,
                    no_lang_auto_detect,
                    agree,
                ]
                error_data = [str(e) if type(e) != str else e for e in error_data]
                print(error_data)
                write_io = StringIO()
                csv.writer(write_io).writerows([error_data])
                csv_upload = write_io.getvalue().encode()

                filename = error_time + "_" + str(uuid.uuid4()) + ".csv"
                print("Writing error csv")
                error_api = HfApi()
                error_api.upload_file(
                    path_or_fileobj=csv_upload,
                    path_in_repo=filename,
                    repo_id="coqui/xtts-flagged-dataset",
                    repo_type="dataset",
                )

                print("Writing error reference audio")
                speaker_filename = error_time + "_reference_" + str(uuid.uuid4()) + ".wav"
                error_api.upload_file(
                    path_or_fileobj=speaker_wav,
                    path_in_repo=speaker_filename,
                    repo_id="coqui/xtts-flagged-dataset",
                    repo_type="dataset",
                )

                space = api.get_space_runtime(repo_id=repo_id)
                if space.stage != "BUILDING":
                    api.restart_space(repo_id=repo_id)
                else:
                    print("TRIED TO RESTART but space is building")
            else:
                if "Failed to decode" in str(e):
                    print("Speaker encoding error", str(e))
                else:
                    print("RuntimeError: non device-side assert error:", str(e))
                return None

        return "output.wav"

In [9]:
prompt_text = "Hello, this is a test."
language_code = "en"
audio_file_path = '/content/1.mp3'  # Path to reference audio file if needed
mic_file_path = None  # Path to microphone recorded audio if needed
use_microphone = False
voice_cleanup = False
disable_lang_auto_detect = False
agree_terms = True

output_file = text_to_speech(prompt_text, language_code, audio_file_path, mic_file_path, use_microphone, voice_cleanup, disable_lang_auto_detect, agree_terms)
if output_file:
    print(f"Audio generated and saved to {output_file}")
else:
    print("Failed to generate audio.")

Generating new audio...
Time to generate audio: 895 milliseconds
Real-time factor (RTF): 0.5381790777811637
Audio saved to output.wav
Audio generated and saved to output.wav
